# Исследование надёжности заёмщиков

Кредитный отдел банка дал задание разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Данные от банка — статистика о платёжеспособности клиентов.

# План работы

- [Изучение входных данных](#изучение)
- [Предобработка данных](#предобработка)
- [Ответы на вопросы банка](#ответы)
- [Общий вывод](#вывод)

## Изучение входных данных <a id='изучение'></a>

In [331]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [332]:
data.groupby('education_id')['education'].value_counts()


education_id  education          
0             высшее                  4718
              ВЫСШЕЕ                   274
              Высшее                   268
1             среднее                13750
              СРЕДНЕЕ                  772
              Среднее                  711
2             неоконченное высшее      668
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                250
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64

In [333]:
data.groupby('family_status_id')['family_status'].value_counts()


family_status_id  family_status        
0                 женат / замужем          12380
1                 гражданский брак          4177
2                 вдовец / вдова             960
3                 в разводе                 1195
4                 Не женат / не замужем     2813
Name: family_status, dtype: int64

In [334]:
data['purpose'].unique()


array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [335]:
data['children'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [336]:
data['debt'].unique()


array([0, 1])

In [337]:
data['gender'].unique()


array(['F', 'M', 'XNA'], dtype=object)

In [338]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

**Вывод**

Открыл датасет. В целом оформление датасета приемлемо, нет необходимости вносить правки в его формат (например, названия колонок). Касательно данных, сразу бросается в глаза следующее: отрицательные значения трудового стажа в днях, а также формат float у значений в этой же колонке, данные даны в часах. Далее разница в регистре у значений в колонке education. Колонка education_id, принцип распределения значений: 0 - высшее, 1 - среднее, 2 - неоконченное высшее, 3 - начальное, 4 - ученая степень.  Принцип распределения значений в family_status_id: 0 - женат/замужем, 1 - гражданский брак, 2 - вдова/вдовец, 3 - в разводе, 4 - не женат/не замужем. Total_income указаны значения в формате float. Колонка purpose: большое количество уникальных значений, которые можно свести к нескольким: недвижимость, образование, автомобиль, свадьба. Колонка children: количество детей варируется от 1 до 5, имеются значения 20 и -1, необходимо рассмотреть эти случаи. Значений в колонке debt всего два - 0 и 1 (либо есть задолженность, либо ее нет, соответсвенно). Колонка gender содержит значение XNA, потребуется замена. 

## Предобработка данных <a id='предобработка'></a>

### Обработка пропусков

In [339]:
data.isna().sum()
data_nan = data[data.isna().any(1)]
#data_nan['education_id'].unique()
#data_nan['family_status_id'].unique()
#data_nan['income_type'].unique()


In [340]:
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] > 30000, 'days_employed'] = data.loc[data['days_employed'] > 30000, 'days_employed'] / 24
print('Пропуски до:', data['days_employed'].isna().sum())
data.loc[data['days_employed'].isna(),'days_employed'] = data['days_employed'].median()
print('Пропуски после:', data['days_employed'].isna().sum())


Пропуски до: 2174
Пропуски после: 0


In [341]:
data['total_income'] = data['total_income'].apply(abs)
data.loc[data['total_income'].isna(),'total_income'] = data['total_income'].median()
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [342]:
data['days_employed'].abs()

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21520     4529.316663
21521    14330.725172
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [343]:
#data['gender'].value_counts()
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [344]:
data['children'] = data['children'].apply(abs)
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

**Вывод**

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

А если бы мы посмотрели на долю этих значений, то смогли бы принять решение в сторону избавления от этих данных ? 
    
</div>

В датасете обнаружены пропуски в колонках days_unemployed, total_income (2174). Пропуск не зависит от уровня образования, хотя нет ни одной строки с пропуском, где уровень образования - Ученая степень. Пропуск также не зависит от семейного положения. Пропуск также не зависит от типа занятости, хотя пропусков нет у таких типов income_type, как студент, в декрете и безработный. Таким образом, можно заключить, что пропуски количественных значений в days_unemployed, total_income не зависит от какой-то группы заемщиков. Возможно, имеется ошибка в выгрузке данных, либо данные не указывались заемщиками. Заполнение пропусков лучше заполнять медианным значением, поскольку разброс значений довольно таки большой. Такую же процедуру выполняем с total_income. В колонке gender единственное значение XNA было заменено на самое часто встречающееся значение F

### Замена типа данных

In [345]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для замены типа данных взял astype('int'). Теперь данные приведены в нужные форматы.

### Обработка дубликатов

In [346]:
print('Дубликаты до:', data.duplicated(keep=False).sum())
duplicated_data = data[data.duplicated(keep=False)].sort_values(by=list(data.columns))
data = data.drop_duplicates(keep=False).reset_index(drop=True)
print ('Дубликаты после:', data.duplicated(keep=False).sum())
data['education'] = data['education'].str.lower()
#data.head(15)
data.info()

Дубликаты до: 106
Дубликаты после: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21419 entries, 0 to 21418
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21419 non-null  int64 
 1   days_employed     21419 non-null  int64 
 2   dob_years         21419 non-null  int64 
 3   education         21419 non-null  object
 4   education_id      21419 non-null  int64 
 5   family_status     21419 non-null  object
 6   family_status_id  21419 non-null  int64 
 7   gender            21419 non-null  object
 8   income_type       21419 non-null  object
 9   debt              21419 non-null  int64 
 10  total_income      21419 non-null  int64 
 11  purpose           21419 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Имеются полные дубликаты, следует их сбросить. Есть неявные дубликаты в колонке education, появляющие из-за разницы регистра. Решение - задать нижний регистр для всех значений колонки.

### Лемматизация

In [347]:
purpose_words = data['purpose'].unique()
purpose = ' '.join(purpose_words)
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(purpose)
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [348]:
def categorize(value):
    cell = m.lemmatize(value)
    if 'автомобиль' in cell:
        return 'автомобиль'
    if 'образование' in cell:
        return 'образование'
    if 'недвижимость' in cell:
        return 'недвижимость'
    if 'жилье' in cell:
        return 'недвижимость'
    if 'свадьба' in cell:
        return 'свадьба'
    return 'прочее'

data['category'] = data['purpose'].apply(categorize)
#data.head(10)
data['category'].value_counts()

недвижимость    10790
автомобиль       4301
образование      4006
свадьба          2322
Name: category, dtype: int64

**Вывод**

В колонке purpose по результатам лемматизации датафрейма выяснилось, что самыми повторяющимися значениями являются: жилье, недвижимость, автомобиль, образовани и свадьба. Жилье и недвижимотсь можно объединить в одну категорию. Соответственно, у нас 4 категории целей на кредитование. 

### Категоризация данных

In [349]:
#data.head(10)
data_numbers = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'category']]
data_numbers

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,category
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...
21414,1,4529,43,1,1,F,компаньон,0,224791,недвижимость
21415,0,14330,67,1,0,F,пенсионер,0,155999,автомобиль
21416,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21417,3,3112,38,1,0,M,сотрудник,1,244093,автомобиль


In [350]:
data_dict_education = data[['education_id', 'education']]
data_dict_education = data_dict_education.drop_duplicates().reset_index(drop=True)
data_dict_education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [351]:
data_dict_family = data[['family_status_id', 'family_status']]
data_dict_family = data_dict_family.drop_duplicates().reset_index(drop=True)
data_dict_family

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Вывод

Категоризацию данных сделал следующую: в отдельные словари выделены колонки с образованием и семейным положением. Наличие дополнительных данных в датафрейме усложняет его изучение. В общем датафрейме лучше оставить только одну колонку с информацией об образовании и семейном положении - id. Пояснение к id лучше вынести в отдельные словари. 

## Ответы на вопросы банка <a id='ответы'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [352]:
return_credit = data[['children', 'debt']]
def children_group (value):
    if value == 0:
        return 'нет детей'
    else:
        return 'есть дети'
return_credit['y_n_children'] = return_credit['children'].apply(children_group)
return_credit.head()

/tmp/ipykernel_36/3473691670.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_credit['y_n_children'] = return_credit['children'].apply(children_group)


,children,debt,y_n_children
0,1,0,есть дети
1,1,0,есть дети
2,0,0,нет детей
3,3,0,есть дети
4,0,0,нет детей


In [353]:
def print_conversion_children_debt (children_list, debt):
    children_debt_func = return_credit[(return_credit['y_n_children'] == children_list) & (return_credit['debt'] == debt)]
    conversion = children_debt_func['debt'].count() / return_credit['debt'].count()
    debt_dict = {0: 'возврат', 1: 'невозврат'}
    print(f'Доля {children_list} и {debt_dict[debt]}: {conversion:.1%}')

children_list = return_credit['y_n_children'].unique()
debt = [0, 1]
for x in children_list:
    for y in debt:
        print_conversion_children_debt(x, y)

Доля есть дети и возврат: 31.2%
Доля есть дети и невозврат: 3.2%
Доля нет детей и возврат: 60.7%
Доля нет детей и невозврат: 5.0%


**Вывод**

По результатам вычислений, можно сделать вывод, что примерно 61% заемщиков не имеют детей и возвращают кредит в срок, при этом 31% заемщиков даже несмотря на наличие детей, все равно возвращают кредит в срок. Как таковой зависимости между наличием детей и возврата кредита в срок - нет. Люди, за исключением малой доли заемщиков, возвращают кредит в срок независимо от того, есть ли дети или их нет

In [354]:
data.pivot_table(
    index='children',
    values='debt'
).round(4) * 100

,debt
children,
0,7.56
1,9.18
2,9.47
3,8.18
4,9.76
5,0.00
20,10.53


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [356]:
family_status_return_debt = data[['family_status_id', 'debt']]
def print_conversion_family_debt (family_list, debt):
    family_debt_func = family_status_return_debt[(family_status_return_debt['family_status_id'] == family_list) & (family_status_return_debt['debt'] == debt)]
    conversion = family_debt_func['debt'].count() / family_status_return_debt['debt'].count()
    debt_dict = {0: 'возврат', 1: 'невозврат'}
    family_list_dict = {0:'женат/замужем', 1:'гражданский брак', 2:'вдовец/вдова', 3:'в разводе', 4:'не женат/не замужем'}
    print(f'Доля {family_list_dict[family_list]} и {debt_dict[debt]}: {conversion:.1%}')

family_list = family_status_return_debt['family_status_id'].unique()
debt = [0, 1]
for x in family_list:
    for y in debt:
        print_conversion_family_debt(x, y)

Доля женат/замужем и возврат: 53.1%
Доля женат/замужем и невозврат: 4.3%
Доля гражданский брак и возврат: 17.6%
Доля гражданский брак и невозврат: 1.8%
Доля вдовец/вдова и возврат: 4.2%
Доля вдовец/вдова и невозврат: 0.3%
Доля в разводе и возврат: 5.2%
Доля в разводе и невозврат: 0.4%
Доля не женат/не замужем и возврат: 11.8%
Доля не женат/не замужем и невозврат: 1.3%


In [357]:
data.pivot_table(
    index='family_status',
    values='debt'
).round(4) * 100

,debt
family_status,
Не женат / не замужем,9.76
в разводе,7.11
вдовец / вдова,6.58
гражданский брак,9.35
женат / замужем,7.56


**Вывод**

Среди общего числа заемщиков, большая доля тех, кто находятся в статусе "женат/замужем", а также тех, кто находятся в гражданском браке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [358]:
income = data[['total_income', 'debt']]
def income_group (value):
    if value <= 30000:
        return 'бедность'
    if value <= 60000:
        return 'средний достаток'
    if value <= 90000:
        return 'состоятельные'
    if value <= 150000:
        return 'богаты'
    return 'сверхбогатый'
income['income_group'] = income['total_income'].apply(income_group)

/tmp/ipykernel_36/1182469301.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['income_group'] = income['total_income'].apply(income_group)


In [361]:
def print_conversion_income_debt (income_list, debt):
    income_debt_func = income[(income['income_group'] == income_list) & (income['debt'] == debt)]
    conversion = income_debt_func['debt'].count() / income['debt'].count()
    debt_dict = {0: 'возврат', 1: 'невозврат'}
    print(f'Доля {income_list} и {debt_dict[debt]}: {conversion:.2%}')

income_list = ['сверхбогатый', 'богаты', 'состоятельные', 'средний достаток',
       'бедность']
debt = [0, 1]
for x in income_list:
    for y in debt:
        print_conversion_income_debt(x, y)

Доля сверхбогатый и возврат: 39.49%
Доля сверхбогатый и невозврат: 3.39%
Доля богаты и возврат: 37.98%
Доля богаты и невозврат: 3.52%
Доля состоятельные и возврат: 10.87%
Доля состоятельные и невозврат: 0.99%
Доля средний достаток и возврат: 3.44%
Доля средний достаток и невозврат: 0.22%
Доля бедность и возврат: 0.09%
Доля бедность и невозврат: 0.01%


In [362]:
income.pivot_table(
    index='income_group',
    values='debt'
).round(4) * 100

,debt
income_group,
бедность,9.09
богаты,8.47
сверхбогатый,7.91
состоятельные,8.38
средний достаток,5.99


**Вывод**

Для облегчения задачи разделим заемщиков по их доходам на несколько категорий. Чем состоятельнее человек - тем больше у него есть возможностей взять кредит и успешно его вернуть

- Как разные цели кредита влияют на его возврат в срок?

In [363]:
purpose = data[['category', 'debt']]
def print_conversion_purpose_debt (category, debt):
    purpose_debt_func = purpose[(purpose['category'] == category) & (purpose['debt'] == debt)]
    conversion = purpose_debt_func['debt'].count() / purpose['debt'].count()
    debt_dict = {0: 'возврат', 1: 'невозврат'}
    print(f'Доля {category} и {debt_dict[debt]}: {conversion:.1%}')

category = ['недвижимость', 'автомобиль', 'образование', 'свадьба']
debt = [0, 1]
for x in category:
    for y in debt:
        print_conversion_purpose_debt(x, y)

Доля недвижимость и возврат: 46.7%
Доля недвижимость и невозврат: 3.7%
Доля автомобиль и возврат: 18.2%
Доля автомобиль и невозврат: 1.9%
Доля образование и возврат: 17.0%
Доля образование и невозврат: 1.7%
Доля свадьба и возврат: 10.0%
Доля свадьба и невозврат: 0.9%


In [364]:
data.pivot_table(
    index='category',
    values='debt'
).round(4) * 100

,debt
category,
автомобиль,9.37
недвижимость,7.25
образование,9.24
свадьба,8.01


**Вывод**

Заемщики в большинстве своем берут кредиты на недвижимость и успешно их возвращают

## Общий вывод <a id='вывод'></a>

Мною была изучена информация в датафрейме, совершены необходимые манипуляции с данными, для облегчения чтения данных и корректной работы с датафреймом. Далее были выполнены задания по обработанному датафрейму, выводы вместе с расчетами находятся после каждого вопроса.